<a href="https://colab.research.google.com/github/okana2ki/intro-to-AI/blob/main/image_recognition3_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import urllib.request
import torch
from torchvision import models, transforms
from PIL import Image

# ImageNetのクラス名をダウンロードします
class_idx = json.load(urllib.request.urlopen('https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'))

# 学習済みのResNet50モデルをロードします
resnet = models.resnet50(pretrained=True)
resnet.eval()

# 入力画像を処理するための変換を定義します
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

while True:
    # ユーザーに画像URLを入力させます
    img_url = input("画像のURLを入力してください（終了するには'q'を入力）：")

    if img_url.lower() == 'q':
        break

    # URLから画像をダウンロードし、メモリ内で開きます
    with urllib.request.urlopen(img_url) as url:
        img = Image.open(url)

    # 画像を変換します
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)

    # モデルを使って画像を推測します
    resnet.eval()
    out = resnet(batch_t)

    # ソフトマックス関数を適用して確率を計算します
    probs = torch.nn.functional.softmax(out, dim=1)[0] * 100

    # 上位3つの予測結果を取得します
    _, indices = torch.topk(probs, 3)
    percentages = probs[indices]

    # 推測結果を表示します
    for idx, percentage in zip(indices, percentages):
        print("クラス:", class_idx[idx.item()], ", 確信度:", percentage.item())

    print('\n' + '=' * 50 + '\n')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 90.3MB/s]


画像のURLを入力してください（終了するには'q'を入力）：https://www5.city.kyoto.jp/zoo/assets/images/mv/04.jpg
クラス: red panda , 確信度: 99.97879028320312
クラス: giant panda , 確信度: 0.010443676263093948
クラス: European polecat , 確信度: 0.006785721052438021


画像のURLを入力してください（終了するには'q'を入力）：q


In [ ]:
import torch
from torchvision import models
from torch.utils.data import DataLoader

# ファインチューニングしたいデータセットをロード
train_dataset = ...
val_dataset = ...

# ResNetモデルをロード
model = models.resnet18(pretrained=True)

# 最終層をファインチューニング用に置き換える
num_classes = len(train_dataset.classes)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# モデルをファインチューニングモードに設定
model.train()

# ファインチューニング用のデータローダーを作成
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# ファインチューニング用の最適化アルゴリズムと損失関数を設定
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

# ファインチューニングの実行
for epoch in range(10):
    for x, y in train_loader:
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()

    # バリデーションデータでモデルを評価
    with torch.no_grad():
        correct = 0
        total = 0
        for x, y in val_loader:
            y_pred = model(x)
            _, predicted = torch.max(y_pred.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()

        accuracy = 100 * correct / total
        print(f'Epoch: {epoch+1}, Accuracy: {accuracy:.2f}%')

# ファインチューニングされたモデルを保存
torch.save(model.state_dict(), 'fine_tuned_resnet18.pt')

ファインチューニング用のデータセットとして動作を確認できる適当なサンプルはありますか？

Colab AI
ファインチューニング用のデータセットとして動作を確認できる適当なサンプルは、以下の通りです。

CIFAR-10
CIFAR-10は、10クラスの画像で構成されるデータセットです。
CIFAR-10は、画像認識の分野で広く使用されているデータセットです。
CIFAR-10は、PyTorchで簡単にロードすることができます。

In [ ]:
from torchvision.datasets import CIFAR10

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
val_dataset = CIFAR10(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())

MNIST
MNISTは、手書き数字の画像で構成されるデータセットです。
MNISTは、画像認識の分野で広く使用されているデータセットです。
MNISTは、PyTorchで簡単にロードすることができます。

In [ ]:
from torchvision.datasets import MNIST

train_dataset = MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
val_dataset = MNIST(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())

これらのデータセットは、どちらもファインチューニング用のデータセットとして動作を確認することができます。

この回答を評価

Natsuki Oka
resnet18をmnistでファインチューニングする、ということですか？

Colab AI
はい、resnet18をmnistでファインチューニングすることは可能です。

しかし、resnet18はもともと大規模な画像認識データセットであるImageNetで学習されたモデルであり、mnistは小さな手書き数字のデータセットであるため、そのままファインチューニングしても十分な性能を発揮できない可能性があります。

そのため、ファインチューニングを行う際には、以下の点に注意する必要があります。

学習率の設定

学習率を小さく設定することで、モデルの重みが大きく更新されるのを防ぎ、過学習を抑制することができます。
ファインチューニングする層の選択

resnet18のすべての層をファインチューニングするのではなく、最後の数層のみをファインチューニングすることで、過学習を抑制することができます。
データ拡張の使用

データ拡張を使用することで、データセットのサイズを人工的に増やし、過学習を抑制することができます。
以下に、resnet18をmnistでファインチューニングするコード例を示します。

In [ ]:
import torch
from torchvision import models
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

# ファインチューニングしたいデータセットをロード
train_dataset = MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
val_dataset = MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

# ResNet18モデルをロード
model = models.resnet18(pretrained=True)

# 最終層をファインチューニング用に置き換える
num_classes = len(train_dataset.classes)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# モデルをファインチューニングモードに設定
model.train()

# ファインチューニング用のデータローダーを作成
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# ファインチューニング用の最適化アルゴリズムと損失関数を設定
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()

# ファインチューニングの実行
for epoch in range(